In [1]:
from gazebo_msgs.srv import SpawnModel,SetModelState
import rospy
from geometry_msgs.msg import Pose, TransformStamped
from gazebo_msgs.msg import ModelState 
from random import randrange
import PyKDL
import numpy as np
import time
import tf
import tf2_ros
import os

    
def Summon_model(model_type):
    
    model_name = model_type[0]
    
    models = ["cylinder", "box", "sphere", "Aerofoil", "Carbon_hood_scoop", "submarine", "Satellite", 
              "Subaru", "roller_empty", "roller_tunnel", "pyramids", "waves", "weird", "SphereN", 
              "big_boat_shell", "SCANIA_D13_Engine", "fordv6_fixed", "ENGINE_L", "HORIZONTAL_R", "engine", 
              "Stator", "Bench", "HUB", "Plate", "CanTool", "BucketTool", "RakeTool", "ShowelTool", 
              "DiscBrake", "CrankShaft", "Piston", "Gearbox", "differential", "Transmission", "Bevelgear", "helical" ]
    
    rot_quat = PyKDL.Rotation.RPY(np.radians(model_type[4]), np.radians(model_type[5]), np.radians(model_type[6])).GetQuaternion() #0.328462 0 0.000036 0 -1.570604 3.141059
    model_pose = Pose()
    model_pose.position.x = model_type[1]
    model_pose.position.y = model_type[2]
    model_pose.position.z =  model_type[3]
    
    model_pose.orientation.x = rot_quat[0]
    model_pose.orientation.y = rot_quat[1]
    model_pose.orientation.z = rot_quat[2]
    model_pose.orientation.w = rot_quat[3]

    spawn_model_client = rospy.ServiceProxy('gazebo/spawn_sdf_model', SpawnModel)
    
    for model in models:
        if model in model_name:
            print(f'Summoning {model}')
            file_name = model

    file_dir = os.path.dirname(os.path.realpath('__file__')) #current dir. path from where this file is executed
    file_path = os.path.join(file_dir, '../sdf_models/')
    
    stat=spawn_model_client( model_name = model_name, model_xml=open(file_path + file_name + '_model.sdf', 'r').read(),
        robot_namespace='', initial_pose=model_pose, reference_frame='world')
    
    if stat.success == False:
        stat=update_model_pose(1, model_name, [model_type[1],model_type[2],model_type[3],model_type[4],model_type[5],model_type[6]])
    
    return stat



def update_model_pose(hold_time, model_name, Task_pose):
    
    model_state_msg = ModelState()
    t_end = time.time() + hold_time

    while True:
        model_state_msg.model_name = model_name
        model_state_msg.pose.position.x = Task_pose[0]
        model_state_msg.pose.position.y = Task_pose[1] 
        model_state_msg.pose.position.z = Task_pose[2]

        model_rot_quat = PyKDL.Rotation.RPY(np.radians(Task_pose[3]), np.radians(Task_pose[4]), np.radians(Task_pose[5])).GetQuaternion()
        model_state_msg.pose.orientation.x = model_rot_quat[0]
        model_state_msg.pose.orientation.y = model_rot_quat[1]
        model_state_msg.pose.orientation.z = model_rot_quat[2]
        model_state_msg.pose.orientation.w = model_rot_quat[3]
        resp = set_state( model_state_msg )
        if(time.time() > t_end):
            break
    return resp 


def publish_tf(model_pose, parent_frame, child_frame):
    static_transformStamped.header.stamp = rospy.Time.now()
    static_transformStamped.header.frame_id = parent_frame
    static_transformStamped.child_frame_id = child_frame

    static_transformStamped.transform.translation.x = model_pose[0]
    static_transformStamped.transform.translation.y = model_pose[1]
    static_transformStamped.transform.translation.z = model_pose[2]

    quat = tf.transformations.quaternion_from_euler(np.radians(model_pose[3]),np.radians(model_pose[4]),np.radians(model_pose[5]))
    static_transformStamped.transform.rotation.x = quat[0]
    static_transformStamped.transform.rotation.y = quat[1]
    static_transformStamped.transform.rotation.z = quat[2]
    static_transformStamped.transform.rotation.w = quat[3]
    broadcaster.sendTransform(static_transformStamped)



rospy.init_node('insert_object',log_level=rospy.INFO)
rospy.wait_for_service('/gazebo/spawn_sdf_model')
set_state = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)

broadcaster = tf2_ros.StaticTransformBroadcaster()
static_transformStamped = TransformStamped()



def multi_objects():
    model_type = ['Plate_'+ str(101), 0.295, 0.329, 0.1, 0, 0, 90]
    Summon_model(model_type)

    model_type = ['Stator_'+ str(101), 0.297, -0.256, 0, 0, 0, 90]
    #model_type = ['CanTool_'+ str(101), 0.453, 0.064, 0, 0, 0, 90]
    Summon_model(model_type)
    
    model_type = ['BucketTool_'+ str(101), 0.319, -0.610, 0.21, 180, 0, 0]
    Summon_model(model_type)
    
    model_type = ['ShowelTool_'+ str(101), 0.202, 0.626, 0.1, 0, 0, 0]
    Summon_model(model_type)


def crank_piston_disk():
    
    model_type = ['DiscBrake_'+ str(101),  0.310,   0.335,   0.13,      0, 0, 0]
    Summon_model(model_type)
    model_type = ['Piston_'+ str(101),     0.332,  -0.111,   0.050,   0, 0, 0]
    Summon_model(model_type)
    
    model_type = ['CrankShaft_'+ str(101), 0.652, 0.398,     0.13,      90, 0, 0]
    Summon_model(model_type)

def gears():
    model_type = ['Bevelgear_'+ str(101), 0.4, -0.15, 0.0283, 102, 90, 90]
    Summon_model(model_type)
    model_type = ['helical_'+ str(101), 0.263, 0.332, 0.1, 0, 0, -90]
    Summon_model(model_type)


print("Loaded!")

Loaded!


### FOR STL models

In [2]:
#model_type = ['Aerofoil_'+ str(101), 1.6, -0.65, 0.1, 0, 0, 0]
#model_type = ['Carbon_hood_scoop_'+ str(101), 0.6, 0, 0.15, 0, 0, 0]

#model_type = ['submarine_'+ str(101), 0.55, 0, 0.0, 0, 0, 90]
#model_type = ['Satellite_'+ str(101), 1.0, -0.8, 0.15, 0, 0, -90]
#model_type = ['roller_empty_'+ str(103),0.4, -0.1, 0.03, 0, 0, 90]
#model_type = ['roller_tunnel_'+ str(103),0.1, 0, 0, 0, 0, 90]
#model_type = ['Subaru_'+ str(101), 0.5, 0, 0.15, 0, 0, 0]
#model_type = ['waves_'+ str(101), 0.8, -0.6, -0.10, 0, 0, 0]
#model_type = ['weird_'+ str(101), 2.1, -0.9, 0, 0, 0, 0]

#model_type = ['Subaru_'+ str(101), 0.4, 0, 0.15, 0, 0, 0]
#model_type = ['weird_'+ str(101), 2.7, -1.4, 0, 0, 0, 0]
#model_type = ['SphereN_'+ str(101), 0.6, 0, 0.25, 0, 0, 0]


#model_type = ['big_boat_shell_'+ str(101), 0.1, -0.4, 0.4, -90, 0, -90]
#model_type = ['SCANIA_D13_Engine_'+ str(101), 1.0, -0.6, 0.4, 0, 0, 90]
#model_type = ['HORIZONTAL_R_'+ str(101), 4.2, 2.45, -0.63, 0, 0, 119]
#model_type = ['ENGINE_L_'+ str(101), 0.8, -16, 2.32, 0, 90, 0]
#model_type = ['fordv6_fixed_'+ str(101), 0.52, 0, 0, 0, 0, 90]


#model_type = ['engine_'+ str(101), 0.15, -0.3, 0.5, -90, 0, 0]
#model_type = ['Stator_'+ str(101), 0.4, 0, 0, 0, 0, 90]

#model_type = ['Bench_'+ str(101), 0.2, 0, 0, 0, 0, -90]

#model_type = ['HUB_'+ str(101), 0.5, 0, 0.1, 0, 0, 90]
#model_type = ['Plate_'+ str(101), 0.3, 0, 0.1, 0, 0, 90]
#model_type = ['CanTool_'+ str(101), 0.1, 0.2, 0, 0, 0, 90]
#model_type = ['BucketTool_'+ str(101), 0.1, -0.6, 0.21, 180, 0, 0]
#model_type = ['RakeTool_'+ str(101), 0.4, -0.3, 0.1, 0, 0, 90]
#model_type = ['ShowelTool_'+ str(101), -0.05, 0.3, 0.1, 0, 0, 0]

#model_type = ['DiscBrake_'+ str(101),  0.310,   0.335,   0.13,      0, 0, 0]
#model_type = ['Piston_'+ str(101),     0.332,  -0.111,   0.050,   0, 0, 0]
#model_type = ['CrankShaft_'+ str(101), 0.652, 0.398,     0.13,      90, 0, 0]



#model_type = ['differential_'+ str(101), 0.4, 0, 0, 90, 0, -90]
#model_type = ['Gearbox_'+ str(101), 0.4, -0.3, 0, 0, 0, 90]

#model_type = ['Transmission_'+ str(101), 0.4, 0, 0, 0, 0, 0]
#model_type = ['Bevelgear_'+ str(101), 0.4, -0.15, 0.0283, 102, 90, 90]
#model_type = ['helical_'+ str(101), 0.263, 0.332, 0.1, 0, 0, -90]






#model_type = ['Bevelgear_'+ str(101), 0.4, -0.15, 0.0283, 102, 90, 90]
#model_type = ['Gearbox_'+ str(101), 0.55, -0.55, 0, 0, 0, 90]
#model_type = ['HORIZONTAL_R_'+ str(101), 4.2, 2.45, -0.63, 0, 0, 119]
#model_type = ['fordv6_fixed_'+ str(101), 0.52, 0, 0, 0, 0, 90]
#Summon_model(model_type)


crank_piston_disk()






#crank_piston_disk()
#gears()



Summoning DiscBrake
Summoning Piston
Summoning CrankShaft


## FOR BOX

In [ ]:
box_name = 'Test_box_'+ str(101)
box_X = 0.5
box_Y = 0.0
box_Z = 0.1

box_R = 0
box_P = 0
box_Yaw = 0

model_type = [model_name,model_X,model_Y,model_Z,model_R,model_P,model_Yaw]

Summon_model(model_type)

### Update Box Pose

In [ ]:
Box_pose = [0.3, 0.0, 0.1, 45., 45., 0.]

parent_frame = "world"
child_frame = "gt_plane"
result = update_model_pose(1, box_name, Box_pose)
publish_tf(Box_pose, parent_frame, child_frame)
result

## FOR CYLINDER

In [ ]:
model_name = 'Test_cylinder_'+ str(101)
model_X = 0.3
model_Y = 0.0
model_Z = 0.1

model_R = 90
model_P = 0
model_Yaw = 90

model_type = [model_name,model_X,model_Y,model_Z,model_R,model_P,model_Yaw]

Summon_model(model_type)

### Update Cylinder Pose

In [ ]:
Cylinder_pose = [0.3, 0.0, 0.1, 90., 0., 90.]

parent_frame = "world"
child_frame = "gt_cylinder"
result = update_model_pose(1, model_name, Cylinder_pose)
publish_tf(Cylinder_pose, parent_frame, child_frame)
result

## FOR SPHERE

In [ ]:
model_name = 'Test_sphere_'+ str(101)
model_X = 0.3
model_Y = 0.0
model_Z = 0.05

model_R = 90
model_P = 0
model_Yaw = 90

model_type = [model_name,model_X,model_Y,model_Z,model_R,model_P,model_Yaw]

Summon_model(model_type)

### Update Sphere Pose

In [ ]:
Capsule_pose = [0.3, 0.0, 0.05, 0., 0., 0.]

parent_frame = "world"
child_frame = "gt_sphere"
result = update_model_pose(1, model_name, Capsule_pose)
publish_tf(Capsule_pose, parent_frame, child_frame)
result